In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv

load_dotenv #take environment variables from .env

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Union[IO[str], NoneType] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Union[str, NoneType] = 'utf-8') -> bool>

In [14]:
KEY=os.getenv("OPENAI_API_KEY")

In [15]:
llm=ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature=0.5)

In [16]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [12]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","level","response_json"],
    template=TEMPLATE
)

In [13]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\joths\anaconda3\envs\GENAI\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [18]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [35]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [36]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [21]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [22]:
file_path="D:\GEN AI Project\data.txt"

In [23]:
file_path

'D:\\GEN AI Project\\data.txt'

In [24]:
with open(file_path,'r') as file:
    TEXT=file.read()
    

In [26]:
print(TEXT)


Machine Learning, often abbreviated as ML, is a subset of artificial intelligence (AI) that focuses on the development of computer algorithms that improve automatically through experience and by the use of data. In simpler terms, machine learning enables computers to learn from data and make decisions or predictions without being explicitly programmed to do so.

At its core, machine learning is all about creating and implementing algorithms that facilitate these decisions and predictions. These algorithms are designed to improve their performance over time, becoming more accurate and effective as they process more data.

In traditional programming, a computer follows a set of predefined instructions to perform a task. However, in machine learning, the computer is given a set of examples (data) and a task to perform, but it's up to the computer to figure out how to accomplish the task based on the examples it's given.

For instance, if we want a computer to recognize images of cats, we

In [37]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [38]:
number=5 
subject="Machine Learning"
tone="simple"

In [39]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": number,
            "subject":subject,
            "tone": tone,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
Machine Learning, often abbreviated as ML, is a subset of artificial intelligence (AI) that focuses on the development of computer algorithms that improve automatically through experience and by the use of data. In simpler terms, machine learning enables computers to learn from data and make decisions or predictions without being explicitly programmed to do so.

At its core, machine learning is all about creating and implementing algorithms that facilitate these decisions and predictions. These algorithms are designed to improve their performance over time, becoming more accurate and effective as they process more data.

In traditional programming, a computer follows a set of predefined instructions to perform a task. However, in machine learning, the computer is given a set of examples (data) and a task to perform, but it's up to the computer to figure out how to accomplish the

In [41]:
response

{'text': "\nMachine Learning, often abbreviated as ML, is a subset of artificial intelligence (AI) that focuses on the development of computer algorithms that improve automatically through experience and by the use of data. In simpler terms, machine learning enables computers to learn from data and make decisions or predictions without being explicitly programmed to do so.\n\nAt its core, machine learning is all about creating and implementing algorithms that facilitate these decisions and predictions. These algorithms are designed to improve their performance over time, becoming more accurate and effective as they process more data.\n\nIn traditional programming, a computer follows a set of predefined instructions to perform a task. However, in machine learning, the computer is given a set of examples (data) and a task to perform, but it's up to the computer to figure out how to accomplish the task based on the examples it's given.\n\nFor instance, if we want a computer to recognize i

In [40]:
print(response)

{'text': "\nMachine Learning, often abbreviated as ML, is a subset of artificial intelligence (AI) that focuses on the development of computer algorithms that improve automatically through experience and by the use of data. In simpler terms, machine learning enables computers to learn from data and make decisions or predictions without being explicitly programmed to do so.\n\nAt its core, machine learning is all about creating and implementing algorithms that facilitate these decisions and predictions. These algorithms are designed to improve their performance over time, becoming more accurate and effective as they process more data.\n\nIn traditional programming, a computer follows a set of predefined instructions to perform a task. However, in machine learning, the computer is given a set of examples (data) and a task to perform, but it's up to the computer to figure out how to accomplish the task based on the examples it's given.\n\nFor instance, if we want a computer to recognize i

In [42]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2493
Prompt Tokens:1678
Completion Tokens:815
Total Cost:0.004147


In [44]:
quiz=response.get('quiz')

In [49]:
quiz=json.loads(quiz)

In [50]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [51]:
quiz_table_data

[{'MCQ': 'What is the main focus of Machine Learning?',
  'Choices': 'a: Improving computer hardware | b: Developing computer algorithms that improve through experience and data | c: Creating new programming languages | d: Enhancing internet speed',
  'Correct': 'b'},
 {'MCQ': 'How does machine learning differ from traditional programming?',
  'Choices': 'a: Machine learning uses predefined instructions | b: Traditional programming uses data to make decisions | c: Machine learning learns from data and improves over time | d: Traditional programming does not involve data',
  'Correct': 'c'},
 {'MCQ': 'What is the main difference between AI and Machine Learning?',
  'Choices': 'a: AI focuses on creating algorithms, while Machine Learning focuses on computer hardware | b: AI mimics human intelligence, while Machine Learning uses data to make predictions | c: AI and Machine Learning are the same thing | d: AI is used for image recognition, while Machine Learning is used for language transl

In [53]:
quiz=pd.DataFrame(quiz_table_data)

In [54]:
quiz.to_csv('Machine Learning Quiz.csv',index=False)

In [46]:
review=response.get('review')

In [47]:
json.loads(review)

{'1': {'mcq': 'What is the main focus of Machine Learning?',
  'options': {'a': 'Improving computer hardware',
   'b': 'Developing computer algorithms that improve through experience and data',
   'c': 'Creating virtual reality environments',
   'd': 'Building robots with human-like intelligence'},
  'correct': 'b'},
 '2': {'mcq': 'What is the core concept of machine learning algorithms?',
  'options': {'a': 'Following predefined instructions',
   'b': 'Learning from data and improving over time',
   'c': 'Mimicking human intelligence',
   'd': 'Performing tasks without data'},
  'correct': 'b'},
 '3': {'mcq': 'In traditional programming, how does a computer perform a task?',
  'options': {'a': 'By learning from data',
   'b': 'By following a set of predefined instructions',
   'c': 'By making predictions',
   'd': 'By mimicking human intelligence'},
  'correct': 'b'},
 '4': {'mcq': 'What is the main difference between AI and machine learning?',
  'options': {'a': 'AI focuses on improv